Shows the data in pitches.csv right now. Hopefully soon will show off a bit more of the data.

In [ ]:
import pandas as pd
import os, datetime
import matplotlib.pyplot as plt

In [ ]:
p_df = pd.read_csv('../input/mlb-game-data/pitches.csv')
p_df.tail()

In [ ]:
# 100+ MPH pitches thrown by the Brewers
p_df[(p_df['Pitching Team'] == 'MIL') & (p_df['MPH'] <= '100') & (p_df['MPH'] != '--')]

In [ ]:
# All the different types of pitches thrown
p_df['Type'].value_counts()

In [ ]:
# gets a bit messy right now...
plt.pie(p_df['Type'].value_counts(), labels=p_df['Type'].value_counts().index.tolist(), autopct='%1.2f%%');

In [ ]:
p_df[p_df['Pitching Team'] == 'MIL']['Pitcher'].value_counts()

In [ ]:
p_df[p_df['Pitcher'] == 'Burnes']['Type'].value_counts()

# Get Pitches Thrown in 2021

In [ ]:
games_df = pd.read_csv('../input/mlb-game-data/games.csv')
games_df.columns

In [ ]:
games_subset = games_df[pd.to_datetime(games_df['Date'], format='%Y-%m-%dT%H:%MZ', utc=True).dt.year == 2021]['Game']
games_subset

In [ ]:
# And here we are
p_df[p_df['Game'].isin(games_subset)]

### To Get CSVs with subset of modified data

In [ ]:
strike_calls = ['Strike Looking', 'Strike Swinging', 'Foul Ball', 
                'Ground Out', 'Fly Out', 'Line Out', 'Pop Out', 
                'Foul Out', 'Bunted Foul', 'Sacrifice Fly', 'Sacrifice', 
                'Batters Fielders Choice (runner Out)', 'Bunt Ground Out', 
                'Bunt Pop Out', 'Strikeout Batter Safe, Passed Ball']

hit_calls = ['Single', 'Double', 'Triple', 'Home Run', 'Ground Rule Double', 
             'Bunt Single', 'Bunt Double', 'Inside The Park Home Run']

ball_calls = ['Ball', 'Hit By Pitch', 'Wild Pitch; Runner Reached', 'Intentional Ball']
error_calls = ['Batter Reached On Error (batter To First)', 'Catchers Interference (batter To First/error)']
other_calls = ['Batters Fielders Choice (all Runners Safe)', 'Batters Interference (batter Out)', 
               'Official Ruling Pending']

def changeOutcome(x):
    if x in strike_calls:
        return 'Strike'
    elif x in hit_calls:
        return 'Hit'
    elif x in ball_calls:
        return 'Ball'
    else:
        return 'Other'

In [ ]:
# could loop this over team abbreviations
# - only other thing to change would be folder the csvs are stored in (in next cell)

team_abbr = 'MIL'

brewers_df = p_df[p_df['Pitching Team'] == team_abbr]

# write pitcher csvs
for pitcher in brewers_df['Pitcher'].unique():
    temp = brewers_df[brewers_df['Pitcher']==pitcher][['Pitch', 'Type', 'MPH', 'Game']]
    # temp['Inning'] = temp['Inning'].apply(lambda x: x.split(' ')[1][:-2])
    
    # to get the pitcher's pitch count in the game
    pitch_counts = []
    gameid = -1
    for i, row in temp.iterrows():
        if gameid != row['Game']:
            pc = 1
            gameid = row['Game']
        else:
            pc += 1
        pitch_counts.append(pc)
    
    temp['Count'] = pitch_counts
    temp['Pitch'] = temp['Pitch'].apply(lambda x: changeOutcome(x))
    # temp[['Pitch', 'Type', 'MPH', 'Count']].to_csv(os.path.join('pitchers', 'brewers', pitcher+'.csv'), index=False, header=False)

In [ ]:
# write list of pitchers
# with open(os.path.join('pitchers', 'brewers', 'pitchers.txt'), 'w') as f: 
#     for pitcher in brewers_df['Pitcher'].unique().tolist():
#         f.write(pitcher+'\n')

In [ ]:
# examples of what the data looks like after being modified
brewers_df[brewers_df['Pitcher']=='Burnes'][['Pitch', 'Type', 'MPH', 'Inning']]

In [ ]:
brewers_df[brewers_df['Pitcher']=='Burnes']['Pitch'].apply(lambda x: changeOutcome(x))